#### 1. Product Reordering Rates by Department and Aisle

In [3]:
import sqlite3
import pandas as pd

db_path = '../data/processed/instacart.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Sample JOIN query as a DataFrame
query = """
SELECT 
    d.department,
    a.aisle,
    COUNT(op.product_id) AS total_orders,
    SUM(op.reordered) AS reorder_count,
    ROUND(SUM(op.reordered) * 1.0 / COUNT(op.product_id), 4) AS reorder_rate
FROM order_products__prior op
JOIN products p ON op.product_id = p.product_id
JOIN aisles a ON p.aisle_id = a.aisle_id
JOIN departments d ON p.department_id = d.department_id
GROUP BY d.department, a.aisle
ORDER BY reorder_rate DESC;
"""

# Read query results into a DataFrame
Product_Reordering_Rates = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Show the resulting DataFrame
Product_Reordering_Rates


,department,aisle,total_orders,reorder_count,reorder_rate
0,dairy eggs,milk,891015,696264,0.7814
1,beverages,water seltzer sparkling water,841533,613977,0.7296
2,produce,fresh fruits,3642188,2615469,0.7181
3,dairy eggs,eggs,452134,318920,0.7054
4,dairy eggs,soy lactosefree,638253,442023,0.6926
...,...,...,...,...,...
129,personal care,beauty,6168,1308,0.2121
130,household,kitchen supplies,9172,1792,0.1954
131,personal care,first aid,10872,2118,0.1948
132,pantry,baking supplies decor,23692,3962,0.1672


#### 2. Order Distribution by Day of Week and Hour


In [4]:
import sqlite3
import pandas as pd

db_path = '../data/processed/instacart.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Sample JOIN query as a DataFrame
query = """
SELECT 
    order_dow,
    order_hour_of_day,
    COUNT(order_id) AS order_count,
    ROUND(COUNT(order_id) * 100.0 / (SELECT COUNT(*) FROM orders), 2) AS percentage
FROM orders
GROUP BY order_dow, order_hour_of_day
ORDER BY order_dow, order_hour_of_day;
"""

# Read query results into a DataFrame
Order_Distribution_by_Day = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Show the resulting DataFrame
Order_Distribution_by_Day


,order_dow,order_hour_of_day,order_count,percentage
0,0,0,3936,0.12
1,0,1,2398,0.07
2,0,2,1409,0.04
3,0,3,963,0.03
4,0,4,813,0.02
...,...,...,...,...
163,6,19,18346,0.54
164,6,20,13392,0.39
165,6,21,10501,0.31
166,6,22,8532,0.25


#### 3. Impact of Days Since Prior Order on Basket Size


In [5]:
import sqlite3
import pandas as pd

db_path = '../data/processed/instacart.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Sample JOIN query as a DataFrame
query = """
SELECT 
    CASE 
        WHEN days_since_prior_order IS NULL THEN 'First Order'
        WHEN days_since_prior_order < 7 THEN 'Less than a week'
        WHEN days_since_prior_order < 14 THEN '1-2 weeks'
        WHEN days_since_prior_order < 21 THEN '2-3 weeks'
        WHEN days_since_prior_order < 28 THEN '3-4 weeks'
        ELSE 'More than 4 weeks'
    END AS order_interval,
    COUNT(DISTINCT o.order_id) AS num_orders,
    ROUND(AVG(basket_size), 2) AS avg_basket_size,
    ROUND(AVG(reorder_ratio), 4) AS avg_reorder_ratio
FROM orders o
JOIN (
    SELECT 
        order_id, 
        COUNT(product_id) AS basket_size,
        SUM(reordered) * 1.0 / COUNT(product_id) AS reorder_ratio
    FROM order_products__prior
    GROUP BY order_id
) bs ON o.order_id = bs.order_id
GROUP BY order_interval
ORDER BY 
    CASE order_interval
        WHEN 'First Order' THEN 0
        WHEN 'Less than a week' THEN 1
        WHEN '1-2 weeks' THEN 2
        WHEN '2-3 weeks' THEN 3
        WHEN '3-4 weeks' THEN 4
        ELSE 5
    END;
"""

# Read query results into a DataFrame
Impact_of_Days = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Show the resulting DataFrame
Impact_of_Days


,order_interval,num_orders,avg_basket_size,avg_reorder_ratio
0,First Order,206209,10.08,0.0000
1,Less than a week,1255259,9.13,0.6725
2,1-2 weeks,907337,11.07,0.6709
3,2-3 weeks,334282,10.81,0.6182
4,3-4 weeks,164841,10.53,0.5809
5,More than 4 weeks,346946,10.09,0.4880


#### 4. Frequently Co-purchased Products

In [6]:
import sqlite3
import pandas as pd

db_path = '../data/processed/instacart.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Sample JOIN query as a DataFrame
query = """
WITH product_pairs AS (
    SELECT 
        a.product_id AS product_1,
        b.product_id AS product_2,
        COUNT(*) AS pair_count
    FROM order_products__prior a
    JOIN order_products__prior b ON a.order_id = b.order_id AND a.product_id < b.product_id
    GROUP BY a.product_id, b.product_id
    HAVING COUNT(*) > 100
)
SELECT 
    p1.product_name AS product_1_name,
    p2.product_name AS product_2_name,
    pp.pair_count
FROM product_pairs pp
JOIN products p1 ON pp.product_1 = p1.product_id
JOIN products p2 ON pp.product_2 = p2.product_id
ORDER BY pp.pair_count DESC
LIMIT 20;
"""

# Read query results into a DataFrame
Frequently_Co_purchased = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Show the resulting DataFrame
Frequently_Co_purchased


,product_1_name,product_2_name,pair_count
0,Bag of Organic Bananas,Organic Hass Avocado,62341
1,Bag of Organic Bananas,Organic Strawberries,61628
2,Organic Strawberries,Banana,56156
3,Banana,Organic Avocado,53395
4,Organic Baby Spinach,Banana,51395
5,Bag of Organic Bananas,Organic Baby Spinach,50372
6,Strawberries,Banana,41232
7,Banana,Large Lemon,40880
8,Organic Strawberries,Organic Hass Avocado,40794
9,Bag of Organic Bananas,Organic Raspberries,40503


#### 5. Customer Segmentation by Shopping Behavior

In [7]:
import sqlite3
import pandas as pd

db_path = '../data/processed/instacart.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Sample JOIN query as a DataFrame
query = """
WITH user_metrics AS (
    SELECT 
        o.user_id,
        COUNT(DISTINCT o.order_id) AS total_orders,
        AVG(basket.basket_size) AS avg_basket_size,
        AVG(basket.reorder_ratio) AS avg_reorder_ratio,
        AVG(o.days_since_prior_order) AS avg_days_between_orders,
        SUM(CASE WHEN o.order_dow IN (0, 6) THEN 1 ELSE 0 END) * 1.0 / 
            COUNT(o.order_id) AS weekend_order_ratio
    FROM orders o
    JOIN (
        SELECT 
            order_id,
            COUNT(product_id) AS basket_size,
            SUM(reordered) * 1.0 / COUNT(product_id) AS reorder_ratio
        FROM order_products__prior
        GROUP BY order_id
    ) basket ON o.order_id = basket.order_id
    WHERE o.eval_set = 'prior'
    GROUP BY o.user_id
    HAVING COUNT(DISTINCT o.order_id) >= 5
)
SELECT 
    CASE 
        WHEN avg_basket_size <= 5 THEN 'Small Basket'
        WHEN avg_basket_size <= 12 THEN 'Medium Basket'
        ELSE 'Large Basket'
    END AS basket_segment,
    CASE 
        WHEN avg_days_between_orders <= 7 THEN 'Frequent Shopper'
        WHEN avg_days_between_orders <= 14 THEN 'Regular Shopper'
        ELSE 'Occasional Shopper'
    END AS frequency_segment,
    CASE 
        WHEN avg_reorder_ratio <= 0.3 THEN 'Low Reorder'
        WHEN avg_reorder_ratio <= 0.6 THEN 'Medium Reorder'
        ELSE 'High Reorder'
    END AS reorder_segment,
    COUNT(*) AS user_count,
    ROUND(AVG(total_orders), 1) AS avg_orders,
    ROUND(AVG(avg_basket_size), 1) AS avg_items_per_order,
    ROUND(AVG(avg_reorder_ratio), 3) AS avg_reorder_rate,
    ROUND(AVG(avg_days_between_orders), 1) AS avg_days_between_orders,
    ROUND(AVG(weekend_order_ratio), 3) AS weekend_order_rate
FROM user_metrics
GROUP BY basket_segment, frequency_segment, reorder_segment
ORDER BY user_count DESC;
"""

# Read query results into a DataFrame
Customer_Segmentation_by_Shopping_Behavior = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Show the resulting DataFrame
Customer_Segmentation_by_Shopping_Behavior


,basket_segment,frequency_segment,reorder_segment,user_count,avg_orders,avg_items_per_order,avg_reorder_rate,avg_days_between_orders,weekend_order_rate
0,Medium Basket,Occasional Shopper,Medium Reorder,20851,10.2,8.5,0.445,19.4,0.343
1,Medium Basket,Regular Shopper,Medium Reorder,16460,17.9,8.5,0.469,10.8,0.324
2,Large Basket,Occasional Shopper,Medium Reorder,14920,9.8,17.2,0.454,19.8,0.360
3,Medium Basket,Regular Shopper,High Reorder,12842,28.4,8.5,0.707,10.0,0.327
4,Large Basket,Regular Shopper,High Reorder,10179,29.2,17.5,0.714,10.0,0.364
5,Medium Basket,Occasional Shopper,Low Reorder,10008,7.4,8.3,0.200,20.3,0.328
6,Large Basket,Regular Shopper,Medium Reorder,8355,14.9,16.6,0.479,10.9,0.358
7,Medium Basket,Frequent Shopper,High Reorder,8024,58.7,8.4,0.740,5.1,0.262
8,Small Basket,Occasional Shopper,Medium Reorder,7858,9.2,3.3,0.463,20.6,0.289
9,Medium Basket,Occasional Shopper,High Reorder,5826,13.8,8.3,0.680,18.3,0.337
